In [80]:
import os
import numpy as np
import pandas as pd
import pickle as pkl
from typing import List

data_path = 'data'

In [37]:
dataset_path = os.path.join(os.pardir, 'dataset.csv')

In [51]:
with open(os.path.join(data_path, 'X_train.pickle'), 'rb') as file:
    X_train = pkl.load(file)

In [39]:
dataset = pd.read_csv(dataset_path)

In [41]:
X_train.shape

(18495, 12, 186)

# Perturbation

In [42]:
from libraries import RNN_get_torch_data as torch_data

In [43]:
train_data = torch_data.get_train_data('data')

In [44]:
X_train = train_data.X
features = train_data.features

In [48]:
X_train.shape

torch.Size([18495, 12, 186])

In [45]:
features = features.astype(str)

In [46]:
features.dtype

dtype('<U40')

In [53]:
categorical = np.logical_or(np.char.startswith(features, 'RC_'),
        np.char.endswith(features, 'Indicator'))[np.newaxis, np.newaxis, :]

In [54]:
perturbation = np.where(categorical, np.zeros_like(X_train), np.random.normal(0, .1, X_train.shape))

In [81]:
def perturbate(X_train: np.ndarray, features: np.ndarray, n_perturbed_replicas: int, seed: int) -> List[np.ndarray]:
    '''
    Inputs:
    ---
    X_train: the array of series to be perturbed.
    features: the features (3rd dimension index) of X_train.
    n_perturbed_replicas: the number of replicas to be added per series.

    Returns:
    ---
    The perturbed array of series.
    '''
    categorical = np.logical_or(np.char.startswith(features, 'RC_'),
        np.char.endswith(features, 'Indicator'))[np.newaxis, np.newaxis, :]
    rng = np.random.Generator(np.random.PCG64(seed))
    noises = rng.normal(0, .1, (n_perturbed_replicas, ) + X_train.shape)
    copies = [X_train]
    for noise in noises:
        perturbation = np.where(categorical, np.zeros_like(X_train), noise)
        copies.append(X_train + perturbation)
    return copies

In [82]:
copies = perturbate(X_train, features, 2)

In [83]:
len(copies)

3